## Compiling Data from HN website

We're gonna be looking at the dataset from Hacker News.

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

We have reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments and then randomly sampling from the remaining submissions.

We're specifically interested in posts with titles that begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. 

Likewise, users submit Show HN posts to show the Hacker News community a project, product, or just something interesting.

In [1]:
from csv import reader
opened_file = open("hacker_news.csv", encoding="utf8")
read_file = reader(opened_file)
hn = list(read_file)

In [2]:
headers = hn[0]
hn = hn[1:]

In [3]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

In [4]:
print('Number of posts in Ask HN:',len(ask_posts))
print('Number of posts in Show HN:',len(show_posts))
print('Number of other posts:',len(other_posts))

Number of posts in Ask HN: 1744
Number of posts in Show HN: 1162
Number of other posts: 17194


In [5]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)

14.038417431192661


In [6]:
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

10.31669535283993


Unsurprisingly ask posts receive more comments on average than show posts.

In [7]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at, num_comments])

In [8]:
counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    date_string = row[0]
    comments_count = row[1]
    date_created = dt.datetime.strptime(date_string, "%m/%d/%Y %H:%M")
    hour_created = date_created.hour
    if hour_created in counts_by_hour:
        counts_by_hour[hour_created] += 1
        comments_by_hour[hour_created] += comments_count 
    else:
        counts_by_hour[hour_created] = 1
        comments_by_hour[hour_created] = comments_count

In [9]:
avg_by_hour = []
for key in counts_by_hour:
    avg_posts = comments_by_hour[key] / counts_by_hour[key]
    avg_by_hour.append([key, avg_posts])

In [10]:
avg_by_hour

[[9, 5.5777777777777775],
 [13, 14.741176470588234],
 [10, 13.440677966101696],
 [14, 13.233644859813085],
 [16, 16.796296296296298],
 [23, 7.985294117647059],
 [12, 9.41095890410959],
 [17, 11.46],
 [15, 38.5948275862069],
 [21, 16.009174311926607],
 [20, 21.525],
 [2, 23.810344827586206],
 [18, 13.20183486238532],
 [3, 7.796296296296297],
 [5, 10.08695652173913],
 [19, 10.8],
 [1, 11.383333333333333],
 [22, 6.746478873239437],
 [8, 10.25],
 [4, 7.170212765957447],
 [0, 8.127272727272727],
 [6, 9.022727272727273],
 [7, 7.852941176470588],
 [11, 11.051724137931034]]

In [11]:
swap_avg_by_hour = []
for key in avg_by_hour:
    swap_avg_by_hour.append([key[1], key[0]])
swap_avg_by_hour

[[5.5777777777777775, 9],
 [14.741176470588234, 13],
 [13.440677966101696, 10],
 [13.233644859813085, 14],
 [16.796296296296298, 16],
 [7.985294117647059, 23],
 [9.41095890410959, 12],
 [11.46, 17],
 [38.5948275862069, 15],
 [16.009174311926607, 21],
 [21.525, 20],
 [23.810344827586206, 2],
 [13.20183486238532, 18],
 [7.796296296296297, 3],
 [10.08695652173913, 5],
 [10.8, 19],
 [11.383333333333333, 1],
 [6.746478873239437, 22],
 [10.25, 8],
 [7.170212765957447, 4],
 [8.127272727272727, 0],
 [9.022727272727273, 6],
 [7.852941176470588, 7],
 [11.051724137931034, 11]]

In [12]:
sorted_swap = sorted(swap_avg_by_hour, reverse = True)

In [13]:
print('Top 5 Hours for Ask Posts Comments')

for row in sorted_swap[:5]:
    hour_formatted = dt.datetime.strptime(str(row[1]), "%H")
    hour_formatted = hour_formatted.strftime("%H:%M")
    
    print('{}: {:.2f} average comments per post'.format(hour_formatted, row[0]))

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post
